## Qube research and technologies data challenge

The goal of this challenge is to try to predict the returns of 50 stocks.

In a classical approach: 
- An autoregressive model could be used.
- An LSTM or attention based model could be used.
- Some manually designed features could be considered and a linear regression using MSE could be used to predict the returns.

However in the scope of this challenge, the following approach was imposed:

- Try to predict the return of a stock if you are given the returns of the $D$ previous days ($D$ being imposed to be equal to 250 in this challenge).
- Thus, your initial features for each stock $S$ and day $t$ would be the returns of the stock in question between the days $t - 1$ and $t - 250$.
- Given a matrix $M \in M_{50 \times T, 250}(\mathbb{R})$ where each line $i$ of this matrix consists of the returns of the stock $S_{i~mod~50}$ from day $\lfloor \frac{i}{50} \rfloor + 250 - 1$ to day $\lfloor \frac{i}{50} \rfloor + 250 - 250$, come up with a new feature space of dimension 10 such that the transformed feature matrix $F = MA$ where $A \in M_{250, 10}(\mathbb(R)$ is a learnable orthonormal matrix. (i.e $AA^{T} = I$).
- Once $F$ is determined, perform a classical linear regression on the new features.

### Useful imports

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from scipy.optimize import minimize, NonlinearConstraint
import sys

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
import torch
from numdifftools import Jacobian
from random import randint
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

### Loading data

In [2]:
X = pd.read_csv('X_train.csv', index_col=0, sep=',')
X.columns.name = 'date'

Y = pd.read_csv('Y_train.csv', index_col=0, sep=',')
Y.columns.name = 'date'

In [3]:
X.head()

date,0,1,2,3,4,5,6,7,8,9,...,744,745,746,747,748,749,750,751,752,753
stocksID,,,,,,,,,,,,,,,,,,,,,
0,-0.018647,-0.013002,-0.010776,-0.016676,-0.005110,0.009092,-0.011745,0.007120,-0.008451,0.009119,...,-0.012525,-0.011716,0.003532,0.009965,0.018142,-0.001236,-0.002732,0.013074,-0.005843,-0.003823
1,-0.008254,-0.022280,0.012173,-0.006820,-0.004055,0.012912,-0.001293,0.009994,-0.002747,0.001664,...,0.014432,-0.002255,-0.011493,0.002291,-0.001346,-0.004026,-0.004672,-0.002889,-0.004984,0.005005
2,-0.008404,-0.013629,-0.006044,-0.003425,-0.009522,-0.001353,-0.000637,0.007640,0.001600,0.007416,...,-0.006245,-0.001329,0.005230,0.003510,0.006022,-0.000343,0.001757,0.004972,0.004916,-0.007338
3,-0.022734,-0.006981,-0.008568,-0.010899,-0.017981,0.002485,-0.011980,0.012446,-0.010636,0.003807,...,-0.005179,-0.003442,0.002733,0.013369,0.019738,0.001201,-0.003669,0.008690,0.000272,-0.008150
4,-0.024546,-0.008315,-0.007991,-0.003515,0.007872,0.007082,-0.004614,-0.008182,-0.005255,0.014404,...,-0.017507,-0.001233,-0.000552,0.004664,0.005202,0.007695,0.003775,0.005097,0.001135,-0.009262


In [4]:
Y.head()

date,250,251,252,253,254,255,256,257,258,259,...,744,745,746,747,748,749,750,751,752,753
stocksID,,,,,,,,,,,,,,,,,,,,,
0,0.001128,-0.001046,-0.007027,-0.009757,-0.005868,-0.008563,-0.005857,-0.004588,-0.000240,0.008705,...,-0.012525,-0.011716,0.003532,0.009965,0.018142,-0.001236,-0.002732,0.013074,-0.005843,-0.003823
1,-0.003274,-0.001070,0.000205,0.004862,-0.012834,-0.004868,0.005476,-0.009097,0.000284,0.002971,...,0.014432,-0.002255,-0.011493,0.002291,-0.001346,-0.004026,-0.004672,-0.002889,-0.004984,0.005005
2,-0.003446,-0.001272,0.005824,-0.006994,-0.005512,-0.003652,0.003997,-0.005139,-0.004550,0.001228,...,-0.006245,-0.001329,0.005230,0.003510,0.006022,-0.000343,0.001757,0.004972,0.004916,-0.007338
3,0.001727,0.003143,-0.003606,-0.022219,-0.017467,-0.004536,-0.003512,-0.000156,0.005489,0.005115,...,-0.005179,-0.003442,0.002733,0.013369,0.019738,0.001201,-0.003669,0.008690,0.000272,-0.008150
4,0.004907,0.002472,-0.001487,0.003317,-0.006636,-0.012548,-0.000942,-0.009569,0.004660,-0.018902,...,-0.017507,-0.001233,-0.000552,0.004664,0.005202,0.007695,0.003775,0.005097,0.001135,-0.009262


### Useful data remodeling

Multi level indexing where:

Let $D_t$ be the matrix of $M_{50, 250}(\mathbb{R})$ which columns are the returns for each of the 50 stocks during the previou 250 days $R_{t-i}$ for $i \in \{ 1,...,250\}$.

Then, we create X_train_reshape to be the vertical concatenation of the $D_t$ matrixes.

Therefore, the features matrix $F$ will be the vertical concatenation of the products $D_t M$ with $M$ being the Stiefel matrix we considered for our model.

In [5]:
X_reshape = pd.concat([ X.T.shift(i+1).stack(dropna=False) for i in range(250) ], axis=1).dropna()
X_reshape.columns = pd.Index(range(1,251), name='timeLag')
Y_reshape = Y.T.stack()

In [6]:
X_reshape.head()

timeLag             1         2         3         4         5         6    \
date stocksID                                                               
250  0         0.000103  0.012387  0.011243  0.002595 -0.008509 -0.002711   
     1        -0.000982  0.003932  0.000050  0.001616 -0.003902 -0.001686   
     2         0.009301  0.003914  0.004995  0.001539  0.001452  0.002809   
     3         0.006515 -0.006553  0.009464  0.005204  0.004227 -0.005438   
     4        -0.006223  0.005415  0.014643  0.005195  0.004489  0.002695   

timeLag             7         8         9         10   ...       241  \
date stocksID                                          ...             
250  0         0.008934  0.006571 -0.018546 -0.008353  ...  0.009119   
     1         0.008810  0.001585 -0.000745 -0.002155  ...  0.001664   
     2         0.005177 -0.006942 -0.013340 -0.008071  ...  0.007416   
     3         0.008861  0.004025 -0.012432 -0.006100  ...  0.003807   
     4         0.007609  0.011437 -0.004804  0.039274  ...  0.014404   

timeLag             242       243       244       245       246       247  \
date stocksID                                                               
250  0        -0.008451  0.007120 -0.011745  0.009092 -0.005110 -0.016676   
     1        -0.002747  0.009994 -0.001293  0.012912 -0.004055 -0.006820   
     2         0.001600  0.007640 -0.000637 -0.001353 -0.009522 -0.003425   
     3        -0.010636  0.012446 -0.011980  0.002485 -0.017981 -0.010899   
     4        -0.005255 -0.008182 -0.004614  0.007082  0.007872 -0.003515   

timeLag             248       249       250  
date stocksID                                
250  0        -0.010776 -0.013002 -0.018647  
     1         0.012173 -0.022280 -0.008254  
     2        -0.006044 -0.013629 -0.008404  
     3        -0.008568 -0.006981 -0.022734  
     4        -0.007991 -0.008315 -0.024546  

[5 rows x 250 columns]

In [7]:
Y_reshape.head()

date  stocksID
250   0           0.001128
      1          -0.003274
      2          -0.003446
      3           0.001727
      4           0.004907
dtype: float64

## Train test splitting

In [8]:
#kf = KFold(n_splits=5, shuffle=True)

### Function to check if a matrix is a Stiefel matrix up to an error factor

In [9]:
def checkOrthonormality(A): 
    
    bool = True
    D, F = A.shape   
    Error = pd.DataFrame(A.T @ A - np.eye(F)).abs()
    
    if any(Error.unstack() > 1e-6):
        bool = False
     
    return bool

### The function below computes the metric that assesses the quality of $A$ and $\beta$ on some data

In [30]:
def metric(A, beta, X, y): 
    
    if not checkOrthonormality(A):
        return -1.0    
    
    Ypred = (X @ A @ beta).unstack().T         
    Ytrue = y
    
    
    Ytrue = Ytrue.div(np.sqrt((Ytrue**2).sum()), 1)    
    Ypred = Ypred.div(np.sqrt((Ypred**2).sum()), 1)

    meanOverlap = (Ytrue * Ypred).sum().mean()

    return  meanOverlap  

### Finding $\beta$ for least square errors regression

In [11]:
def fitBeta(A, X, y):
    
    predictors = X @ A
    targets = y
    beta = np.linalg.inv(predictors.T @ predictors) @ predictors.T @ targets
    
    return beta.to_numpy()

## First idea: Convex optimization

We could consider this problem as a pure optimization problem with constraints over $A$.

We know that our predictions are given by the formula (here $X = X_{reshape}$):

$Y_{pred} = XA\beta$

$XA = (\sum_{0 \leq l \leq 249} X_{il} A_{lj})_{ij}$

let $C = XA$

then:

$XA\beta = C\beta = (\sum_{0 \leq j \leq 9} C_{ij} \beta_{j})_{i} = (\sum_{0 \leq j \leq 9} \sum_{0 \leq l \leq 249} X_{il} A_{lj} \beta_{j})_{i}$

Therefore using the mean squared error as our loss function, we arrive at the following optimization problem:

$argmin_{A, \beta}\frac{1}{25200}\sum_{0 \leq i \leq 24199} (\sum_{0 \leq j \leq 9} \sum_{0 \leq l \leq 249} X_{il} A_{lj} \beta_{j} - y_{i})^{2}$

with the constraint that $A^TA = I$

that we could formalize as follows:

$\forall i \in \{0,..,9\}\forall j \in \{0,..,9\} \sum_{0 \leq l \leq 249}A_{li}A_{lj} = \delta_{ij}$

As much as I would love to introduce the lagrange multipliers and perform a stochastic gradient descent by hands. The task seems tedious. Therefore, I will look into some optimization libraries in order to solve for my parameters.

Little optimization problem:
It seems that when optimizing, the algorithm converges to a minimum that's not a point in the feasible set.(Though we could argue it's close enough to be in it). We take that point and shift it a little bit to be in the feasible by autonormalizing A and fitting beta.

### Defining the loss function

In [12]:
def loss_func(x):
    """
        Let's note that x is a 1D vector where:
        - the first 10 elements will be the elements of beta (x[:10])
        - the next 250 * 10 elements will be the elements of A flattened row wise.
        (meaning that x[10] is the first column of the first line of A and x[20] is the
        first column of the second row of A)
    """
    pseudo_beta = x[:10]
    pseudo_A = np.reshape(x[10:], (250, 10))
    Ypred = (X_reshape @ pseudo_A @ pseudo_beta).unstack().T         
    Ytrue = Y_reshape
    return ((Ypred - Ytrue)**2).sum().sum()


### Defining our contraints

In [13]:
list_cons = []
for i in range(9):
    for j in range(i, 9):
        list_cons.append(NonlinearConstraint(lambda x: sum([x[10+l*10+i]*x[10+l*10+j] for l in range(249)])\
                                             - float(i == j), -1e-6, 1e-6))

### Define our starting optimization points

We choose to start with a decent feasible solution. Since the constraints are on $A$, we choose to initialize $A$ with the matrix composed of the 10 first principal components of $M$.

In [20]:
pca = PCA()
scaler = StandardScaler()
pca.fit(scaler.fit_transform(X_reshape))
begin_A = pca.components_[0:10].T
begin_A = begin_A.reshape((10*250, ))

In [21]:
begin_beta = np.array([0.3 for _ in range(10)])

### Optimization process

In [22]:
res_opt = minimize(loss_func, np.concatenate((begin_beta, begin_A)), method ='SLSQP')

KeyboardInterrupt: 

### Retrieve solution

In [ ]:
res = res_opt.x
opt_beta = res[:10]
opt_A = np.reshape(res[10:], (250, 10))

### Make the solution feasible

NB : We didnt include the constraints in the optimizer because they slowed the optimization process without even leading to a final feasible solution.

In [ ]:
A = np.linalg.qr(opt_A)[0]
beta = fitBeta(opt_A, X_reshape, Y_reshape)

In [ ]:
metric = metric_train(A, beta, X_reshape, Y_reshape)
print(metric)

## Second idea: Using an MLP and the Adam optimizer
### Defining the model to be optimized

The model will be a multilayer perceptron where the first layer is the learnable Stiefel matrix.

In [23]:
class QrtModel(torch.nn.Module):
    def __init__(self):
        super(QrtModel, self).__init__()
        self.A = torch.nn.Linear(in_features=250, out_features=10, bias=False)
        self.beta = torch.nn.Linear(in_features=10, out_features=1, bias=False)
    def forward(self, x):
        new_features = self.A(x)
        final_pred = self.beta(new_features)
        return final_pred

## Optimizing the considered neural network

In [24]:
full_train_data = X_reshape.merge(Y_reshape.rename("y"), left_index=True, right_index=True)
full_train_data_to_tensor = torch.tensor(full_train_data.values)
batches_iterator = torch.utils.data.DataLoader(full_train_data_to_tensor, batch_size=50)
qrt_model = QrtModel()
opt = torch.optim.Adam(qrt_model.parameters(), lr=0.00001)
loss_function = torch.nn.MSELoss()
N_EPOCHS = 150
for epoch in range(N_EPOCHS):
    for batch in batches_iterator:
        x_batch = batch[:, :-1].to(torch.float32)
        y_batch = batch[:, -1].to(torch.float32)
        opt.zero_grad()
        pred = qrt_model(x_batch)
        n_lines = pred.shape[0]
        pred = torch.reshape(pred, (n_lines,))
        loss = loss_function(pred, y_batch)
        loss.backward()
        opt.step()
for param in qrt_model.A.parameters():
    A = np.transpose(param.detach().numpy())
    A = np.linalg.qr(A)[0]

#for param in qrt_model.beta.parameters():
#    beta = param.detach().numpy()[0]

beta = fitBeta(A, X_reshape, Y_reshape)

In [31]:
print(metric(A, beta, X_reshape, Y))

0.12945404773905905


### Transform the shape of the matrix $A$ and $\beta$ to the shape requested in the challenge

In [36]:
def parametersTransform(A, beta, D=250, F=10):
    
    if A.shape != (D, F):
        print('A has not the good shape')
        return
    
    if beta.shape[0] != F:
        print('beta has not the good shape')
        return        
    
    output = np.hstack( (np.hstack([A.T, beta.reshape((F, 1))])).T )
    
    return output

### puts the output in a csv file to be later submitted

In [39]:
# from output to csv file...
output = parametersTransform(A, beta)
pd.DataFrame(output).to_csv('submission.csv')